In [ ]:
import numpy as np
import pandas as pd
import preprocessor
from predict import predict
from keras.models import load_model

In [ ]:
# Normalize data for model input
def normalize(data):
    columns_to_normalize = ufc_data.columns
    columns_to_normalize = columns_to_normalize.drop(["Name", "Stance", "DOB", "Name.1", "Stance.1", "DOB.1", "Win"])
    ufc_data[columns_to_normalize] = ufc_data[columns_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    ufc_data[columns_to_normalize] = ufc_data[columns_to_normalize].fillna(0)
    return ufc_data[columns_to_normalize].to_numpy()

In [ ]:
def present(pair, opposite):
    print(pair)
    for i in range(len(opposite)):
        if pair[0] in opposite.iloc[i].values and pair[1] in opposite.iloc[i].values:
            print("######### PRESENT ##########")
            return True
    
    return False

In [ ]:
def arr_compare(fights, odds):
    drop_fights = []
    drop_odds = []

    print("Checking if fights in odds")
    for i in range(len(fights)):
        if not present([fights.iloc[i]["Name"], fights.iloc[i]["Name.1"]], odds):
            # print(f"Dropping from fights: {i}")
            drop_fights.append(i)

    print("Checking if odds in fights")
    for i in range(len(odds)):
        if not present([odds.iloc[i]["R_fighter"], odds.iloc[i]["B_fighter"]], fights):
            # print(f"Dropping from odds: {i}")
            drop_odds.append(i)
    
    if len(drop_fights) != 0:
        fights = fights.drop(drop_fights)
    
    if len(drop_odds) != 0:
        odds = odds.drop(drop_odds)
    
    return (fights, odds)


In [ ]:
# Load actual odds
odds = pd.read_csv("UFC-data/fightodds.csv")

# Load fighter odds
name = "Conor McGregor"
fights_as_r = odds.loc[odds["R_fighter"] == name].drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
fights_as_b = odds.loc[odds["B_fighter"] == name].drop(["Unnamed: 0", "Unnamed: 0.1"], axis=1)
fight_odds = pd.concat([fights_as_r, fights_as_r])

In [ ]:
# Load all fight metrics for that fighter
ufc_data = preprocessor.hoe_encode(preprocessor.load(name))
x = normalize(ufc_data)

In [ ]:
ufc_data = ufc_data.sort_values(by=["Name", "Name.1"])
fight_odds = fight_odds.sort_values(by=["R_fighter", "B_fighter"])

ufc_data.reset_index(drop=True, inplace=True)
fight_odds.reset_index(drop=True, inplace=True)

print(ufc_data.index)
print(fight_odds.index)

ufc_data, fight_odds = arr_compare(ufc_data, fight_odds)

print(len(ufc_data))
print(len(fight_odds))